# FMLIKH Model: Startup Shear

## Learning Objectives

1. Fit FMLIKH to **startup shear** data with multi-mode response
2. Analyze **multi-mode stress overshoot** features
3. Decompose transient response into mode contributions
4. Compare FMLIKH vs single-mode FIKH predictions

## Prerequisites

- NB07: FMLIKH Flow Curve
- NB02: FIKH Startup Shear (single-mode concepts)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

sys.path.insert(0, os.path.join("..", "utils"))
from fikh_tutorial_utils import (
    load_pnas_startup, save_fikh_results, print_convergence_summary,
    compute_fit_quality, get_fmlikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings("ignore", category=FutureWarning)

## 2. Load Data

In [ ]:
GAMMA_DOT = 1.0
time_data, stress_data = load_pnas_startup(gamma_dot=GAMMA_DOT)
strain_data = GAMMA_DOT * time_data

print(f"Data points: {len(time_data)}")
print(f"Shear rate: {GAMMA_DOT} 1/s")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title(f"Startup Shear Data (γ̇ = {GAMMA_DOT} 1/s)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 3. NLSQ Fitting

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="startup", strain=strain_data)
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nMode info:")
for m in model.get_mode_info()["modes"]:
    print(f"  Mode {m['mode']}: G={m['G']:.4g}, τ={m['tau']:.4g}")

In [ ]:
time_fine = np.linspace(0.01, time_data.max(), 300)
strain_fine = GAMMA_DOT * time_fine
stress_pred = model.predict(time_fine, test_mode="startup", strain=strain_fine)

metrics = compute_fit_quality(stress_data, model.predict(time_data, test_mode="startup", strain=strain_data))

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress_data, "ko", markersize=5, alpha=0.7, label="Data")
ax.plot(time_fine, stress_pred, "-", lw=2.5, color="C0", label=f"FMLIKH ({N_MODES} modes)")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title(f"FMLIKH Startup Fit (R² = {metrics['R2']:.5f})")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model.parameters.get_value(n) for n in param_names if n in model.parameters}

NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

t0 = time.time()
result = model.fit_bayesian(
    time_data, stress_data, test_mode="startup", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure", "sigma_y0"]
print_convergence_summary(result, key_params)

## 5. Save Results

In [ ]:
save_fikh_results(model, result, "fmlikh", "startup", param_names)

## Key Takeaways

1. **Multi-mode captures complex overshoot** behavior
2. **Each mode contributes** at its characteristic time scale
3. **Shared alpha** maintains fractional memory across modes
4. **More modes** → better fit but more parameters